# Data Acquisition

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder.getOrCreate()

## Exercise 1
This exercises uses the case.csv, dept.csv, and source.csv files from the san antonio 311 call dataset.

Read the case, department, and source data into their own spark dataframes.

In [7]:
case = spark.read.csv("data/case.csv", header=True, inferSchema=True)

In [6]:
dept = spark.read.csv("data/dept.csv", header=True, inferSchema=True)

In [5]:
source = spark.read.csv("data/source.csv", sep=",", header=True, inferSchema=True)

## Exercise 2
Let's see how writing to the local disk works in spark:

Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
Inspect your folder structure. What do you notice?

## Exercise 3
Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.